# Workbench

The goal of this project is to develop a PyShiny app capable of visualising the following for a selected neighbourhood of Boston, MA:

1. The most common **destination** for cyclists renting a BlueBike in that neighbourhood
2. The most common **origin** for cyclists who rented a BlueBike elsewhere, and finished their journey in the selected neighborhood
3. Over the course of the day, the times at which the most cyclists *arrived* in the neighbourhood by bicycle
4. Over the course of the day, the times at which the most cyclists *departed* the neighbourhood by bicycle

Ideally, the following variables will be selectable in the user input:
- The neighbourhood
- The direction (inbound vs. outbound)

The data processing will be carried out using a local instance of PySpark

## 1. Data engineering and processing
### 1. Setting up Spark

We start by setting up a PySpark cluster

In [1]:
#Reference: https://sparkbyexamples.com/spark/how-to-create-a-sparksession-and-spark-context/
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('SparkByExamples.com') \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 18:26:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


We then check that it runs correctly

In [3]:
print("First SparkContext:");
print("APP Name :"+spark.sparkContext.appName);
print("Master :"+spark.sparkContext.master);

First SparkContext:
APP Name :SparkByExamples.com
Master :local[1]


### 2. Accessing the BlueBike data
First, we configure the environment zip path variables

The first approach was not successful

In [4]:
import os

home_dir =os.environ['zip_path']

KeyError: 'zip_path'

It was necessary to use the load_dotenv package:

In [5]:
#Location: https://stackoverflow.com/questions/63019506/python-get-value-of-env-variable-from-a-specific-env-file
from dotenv import load_dotenv
load_dotenv()
import os
x = os.getenv("zip_path")
print(x)

https://s3.amazonaws.com/hubway-data/202211-bluebikes-tripdata.zip


### 3. Transforming and storing the data locally
### 4. Checking retrieval of the data
## 2. PyShiny development

I am a working Jupyter cell. Hooray!
